<a href="https://colab.research.google.com/github/Orneyfish/Orney/blob/master/BERT_SENT_EMB_Medical_SEARCH_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
! pip install bert-serving-server bert-serving-client
! pip install tensorflow-gpu==1.14.0-rc0

     |████████████████████████████████| 71kB 2.6MB/s 
  Created wheel for GPUtil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7413 sha256=0977c37a1f3a6ccbed0c2533afe04c73e0de113a6d77f6774f1f718029da5640
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built GPUtil
ERROR: Could not find a version that satisfies the requirement tensorflow-gpu==1.14.0-rc0 (from versions: 0.12.1, 1.0.0, 1.0.1, 1.1.0, 1.2.0, 1.2.1, 1.3.0, 1.4.0, 1.4.1, 1.5.0, 1.5.1, 1.6.0, 1.7.0, 1.7.1, 1.8.0, 1.9.0, 1.10.0, 1.10.1, 1.11.0, 1.12.0, 1.12.2, 1.12.3, 1.13.1, 1.13.2, 1.14.0, 1.15.0rc0, 1.15.0rc1, 1.15.0rc2, 1.15.0rc3, 1.15.0, 1.15.2, 2.0.0a0, 2.0.0b0, 2.0.0b1, 2.0.0rc0, 2.0.0rc1, 2.0.0rc2, 2.0.0, 2.0.1, 2.1.0rc0, 2.1.0rc1, 2.1.0rc2, 2.1.0, 2.2.0rc0, 2.2.0rc1, 2.2.0rc2, 2.2.0rc3, 2.2.0rc4)
ERROR: No matching distribution found for tensorflow-gpu==1.14.0-rc0


In [6]:
# Install the library using pip
!pip install sentence-transformers

     |████████████████████████████████| 61kB 2.3MB/s 
     |████████████████████████████████| 573kB 8.3MB/s 
     |████████████████████████████████| 890kB 29.1MB/s 
     |████████████████████████████████| 3.7MB 44.3MB/s 
     |████████████████████████████████| 1.0MB 36.0MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.2.6.1-cp36-none-any.whl size=74031 sha256=39e465ca56cde859d9f234d94937ab7d399d7330d1b273c6fca28ace4737d460
  Stored in directory: /root/.cache/pip/wheels/d7/fa/17/2b081a8cd8b0a86753fb0e9826b3cc19f0207062c0b2da7008
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=8972eb37aa8be5f06dbb0a9762892afb446cb9eea950601b25c2cb75e3241221
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sentence-transformers sacremoses


In [7]:
from sentence_transformers import SentenceTransformer

# Load the BERT model. Various models trained on Natural Language Inference (NLI) https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/nli-models.md and 
# Semantic Textual Similarity are available https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/sts-models.md

model = SentenceTransformer('bert-base-nli-mean-tokens')

100%|██████████| 405M/405M [00:18<00:00, 21.6MB/s]


In [42]:

import pandas as pd

df = pd.read_csv("/content/metadata_mod_1.csv", delimiter='\t', header=None, names=['title'],  encoding='utf-8')
df = df.replace(',','', regex=True)
# Report the number of sentences.
print('Number of training sentences: {:,}\n'.format(df.shape[0]))

# Display 10 random rows from the data.
df.sample(10)

Number of training sentences: 57,382



,title
21507,Ganglioside-Linked Terminal Sialic Acid Moieti...
19932,Health care policy makers we have a problem
30370,Heterologous virus peptide immunogen productio...
15976,Protein microarrays for systems biology
37670,Tracking progress towards malaria elimination ...
55710,Novel coronavirus: From discovery to clinical ...
924,Computational Resources in Infectious Disease:...
34848,Deciphering the power of isolation in controll...
56664,Posttraumatic stress disorder in convalescent ...
57281,A habitat-based model for the spread of hantav...


In [43]:
df.sample(5)[['title']]

,title
30224,Contents to volume 14
45058,Camphor-based symmetric diimines as inhibitors...
43682,Regulation of t cell responses during central ...
55122,Necrotizing Enterocolitis and Infection
18696,A differential proteome in tumors suppressed b...


In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# Get the lists of sentences and their labels.
sentences = df.title.values

In [45]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [46]:
# Print the original sentence.
print(' Original: ', sentences[0])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(sentences[0]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))

 Original:  Sequence requirements for RNA strand transfer during nidovirus discontinuous subgenomic RNA synthesis
Tokenized:  ['sequence', 'requirements', 'for', 'rna', 'strand', 'transfer', 'during', 'ni', '##do', '##virus', 'disco', '##nti', '##nu', '##ous', 'sub', '##gen', '##omic', 'rna', 'synthesis']
Token IDs:  [5537, 5918, 2005, 12987, 11226, 4651, 2076, 9152, 3527, 23350, 12532, 16778, 11231, 3560, 4942, 6914, 22026, 12987, 10752]


In [50]:
# A corpus is a list with documents split by sentences.

# Each sentence is encoded as a 1-D vector with 78 columns
sentence_embeddings = model.encode(sentences)

print('Sample BERT embedding vector - length', len(sentence_embeddings[0]))

print('Sample BERT embedding vector - note includes negative values', sentence_embeddings[0])

Sample BERT embedding vector - length 768
Sample BERT embedding vector - note includes negative values [-0.07213004  0.49508727 -0.07514993 -0.28801617  0.5327266  -0.4858256
  0.45519215 -0.18853423  0.32917264  0.01886304 -0.20000336 -0.06736126
 -0.07661465  0.7415834  -0.34379065  0.6793352  -0.74183947 -0.20254117
  0.48773494 -0.07760305 -0.71833897  0.3655062  -0.7240679  -0.19259787
  0.25161403  0.41441235  0.77206737 -0.5407673  -0.5124387  -0.18588088
 -0.12911932 -0.18894884  0.30665416  0.14398792 -0.24391262  0.2224096
 -0.47064054  0.04134032  0.51653165  0.4352095  -0.27152145 -0.72647464
 -0.1206138   0.21875894 -0.3527094  -0.9090545   0.5202691   1.0837411
 -0.785386   -0.24086566 -0.38559896 -0.4720218   1.4485849   0.5541864
  0.28284058 -0.34595186  0.50730276 -0.8881728  -0.03555429 -0.23354387
 -0.0408203  -0.2783789  -0.02273689  0.803502   -0.17334306  0.358125
 -0.04606174 -0.5939245  -1.2421298  -1.369909    0.1677198  -0.45662445
 -0.87887335 -0.16424954  0

In [51]:
!pip install sklearn
from sklearn.metrics.pairwise import cosine_similarity
import scipy

In [55]:
query = 'HydroxyCholroquinone' #@param {type: 'string'}

queries = [query]
query_embeddings = model.encode(queries)

# Find the closest sentences of the corpus for each query sentence based on cosine similarity
number_top_matches =  5#@param {type: "number"}

print("Semantic Search Results")

for query, query_embedding in zip(queries, query_embeddings):
    distances = scipy.spatial.distance.cdist([query_embedding], sentence_embeddings, "cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for idx, distance in results[0:number_top_matches]:
        print(sentences[idx].strip(),  "(Cosine Score: %.4f)" % (1-distance))

Semantic Search Results




Query: HydroxyCholroquinone

Top 5 most similar sentences in corpus:
Pharmacology of Chloroquine and Hydroxychloroquine (Cosine Score: 0.8318)
Chloroquine or hydroxychloroquine for prophylaxis of COVID-19 (Cosine Score: 0.8263)
Hydroxychloroquine and â€œoff-labelâ€ utilization in the treatment of oral conditions (Cosine Score: 0.8070)
Nicotine and the renin-angiotensin system (Cosine Score: 0.7832)
Synthesis of a-secomethylenamino- and substituted amidoximotriterpenoids (Cosine Score: 0.7750)
